In [1]:
import os
import pandas as pd
import requests
import bs4
from bs4 import BeautifulSoup
import nltk
from nltk import word_tokenize
from nltk import sent_tokenize
import re

In [2]:
nltk.download("punkt")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [60]:
pd.set_option('display.max_colwidth', None)
df= pd.read_excel("Input.xlsx")
df.head()

,URL_ID,URL
0,123.0,https://insights.blackcoffer.com/rise-of-telemedicine-and-its-impact-on-livelihood-by-2040-3-2/
1,321.0,https://insights.blackcoffer.com/rise-of-e-health-and-its-impact-on-humans-by-the-year-2030/
2,2345.0,https://insights.blackcoffer.com/rise-of-e-health-and-its-imapct-on-humans-by-the-year-2030-2/
3,4321.0,https://insights.blackcoffer.com/rise-of-telemedicine-and-its-impact-on-livelihood-by-2040-2/
4,432.0,https://insights.blackcoffer.com/rise-of-telemedicine-and-its-impact-on-livelihood-by-2040-2-2/


In [6]:
def extract_text_from_link(link):
    # Extracts text from the HTML content of the given link.

    response = requests.get(link)
    soup = BeautifulSoup(response.content, "html.parser")
    article_title=soup.find("title").text

    article_body=soup.find("div", class_="td-post-content tagdiv-type")
    if article_body is None or not article_body.text.strip():
        article_body = soup.find("div", class_="tdb-block-inner td-fix-index")
    if article_body and article_body.text.strip():
        paragraphs = article_body.find_all("p")
        if paragraphs:
            article_body = '\n\n'.join(paragraph.get_text(strip=True) for paragraph in paragraphs)
    #article='\n'.join([article_title, article_body])
    article=str(article_title) + " " + str(article_body) #if article_title and article_body else " "
    return article
    #return article_title, article_body

In [7]:
def extract_text_from_link2(link):
    # Extracts text from the HTML content of the given link.

    response = requests.get(link)
    soup = BeautifulSoup(response.content, "html.parser")
    article_title=soup.find("title").text

    article_body=soup.find("div", class_="td-post-content tagdiv-type")
    if article_body is None or not article_body.text.strip():
        article_body = soup.find("div", class_="tdb-block-inner td-fix-index")
    if article_body:
        paragraphs = article_body.find_all("p")
        if paragraphs:
            article_body = '\n\n'.join(paragraph.get_text(strip=True) for paragraph in paragraphs)
    return article_title, article_body

In [63]:
 # Extract the text from the links.
 text_list = []
 for link in df["URL"]:
  #text=extract_article_text(link)
  text = extract_text_from_link(link)
  text_list.append(text)

# Add the text column to the DataFrame.
df["text"] = text_list

In [38]:
df.head(10)

URL_ID  \
0    123.0   
1    321.0   
3   4321.0   
4    432.0   
5   2893.8   
6   3355.6   
7   3817.4   
9   4741.0   
10  5202.8   
11  5664.6   

                                                                                                                                     URL  \
0                                        https://insights.blackcoffer.com/rise-of-telemedicine-and-its-impact-on-livelihood-by-2040-3-2/   
1                                           https://insights.blackcoffer.com/rise-of-e-health-and-its-impact-on-humans-by-the-year-2030/   
3                                          https://insights.blackcoffer.com/rise-of-telemedicine-and-its-impact-on-livelihood-by-2040-2/   
4                                        https://insights.blackcoffer.com/rise-of-telemedicine-and-its-impact-on-livelihood-by-2040-2-2/   
5                                 https://insights.blackcoffer.com/rise-of-chatbots-and-its-impact-on-customer-support-by-the-year-2040/   
6                                           https://insights.blackcoffer.com/rise-of-e-health-and-its-imapct-on-humans-by-the-year-2030/   
7                                                https://insights.blackcoffer.com/how-does-marketing-influence-businesses-and-consumers/   
9                                                             https://insights.blackcoffer.com/negative-effects-of-marketing-on-society/   
10                                                        https://insights.blackcoffer.com/how-advertisement-marketing-affects-business/   
11  https://insights.blackcoffer.com/rising-it-cities-will-impact-the-economy-environment-infrastructure-and-city-life-by-the-year-2035/   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    

In [10]:
# Create a folder to save articles
if not os.path.exists("articles"):
    os.mkdir("articles")


# Loop through the dataframe rows and extract text from links
for index, row in df.iterrows():
    url_id = row['URL_ID']
    url = row['URL']
    article_title, article_body = extract_text_from_link2(url)

    # Save extracted content to a text file in the "articles" folder
    filename = os.path.join("articles", f"{url_id}.txt")
    with open(filename, 'w', encoding='utf-8') as file:
        file.write(f"Title: {article_title}\n\n")
        file.write(f"Article Body:\n{article_body}\n")



In [18]:
# Create a new column to store article content
df["article_content"] = ""
# Loop through the dataframe rows and read saved articles
for index, row in df.iterrows():
    url_id = row['URL_ID']
    filename = os.path.join("articles", f"{url_id}.txt")

    if os.path.exists(filename):
        with open(filename, 'r', encoding='utf-8') as file:
            article_content = file.read()
            df.at[index, "article_content"] = article_content

In [64]:
# Clean the data
import re
df['text']=df['text'].replace('\n', ' ', regex=True)
def preprocess_text(text):
  #text = re.sub('\n', ' ', text)
  special_characters = ["|", "&"]
  text = re.sub('[| & #]', ' ', text)
  #text=re.sub('&', ' ', text)
  #text = re.sub('[^a-zA-Z ]', ' ', text)
  text = re.sub('[^A-Za-z0-9 ./ ]+', ' ', text)
  # Remove extra spaces
  text = re.sub(' +', ' ', text)
  return text
df['text'] = df['text'].apply(preprocess_text)

In [ ]:
df.head()

In [12]:
!unzip /content/StopWords-20230805T175247Z-001.zip

Archive:  /content/StopWords-20230805T175247Z-001.zip
  inflating: StopWords/StopWords_Generic.txt  
  inflating: StopWords/StopWords_Currencies.txt  
  inflating: StopWords/StopWords_Auditor.txt  
  inflating: StopWords/StopWords_DatesandNumbers.txt  
  inflating: StopWords/StopWords_GenericLong.txt  
  inflating: StopWords/StopWords_Geographic.txt  
  inflating: StopWords/StopWords_Names.txt  


In [65]:
custom_stop_words_folder = "/content/StopWords"

custom_stop_words = set()

# Iterate through the files in the folder
for filename in os.listdir(custom_stop_words_folder):
    file_path = os.path.join(custom_stop_words_folder, filename)
    with open(file_path, 'r', encoding='latin-1') as file:
        words = file.read().splitlines()
        custom_stop_words.update(words)


# Convert stop words to lowercase for case-insensitive matching
custom_stop_words = set(word.lower() for word in custom_stop_words)


In [41]:
len(custom_stop_words)

12738

In [66]:
def clean_text_with_custom_stop_words(text, custom_stop_words):
    words = text.split()  # Split text into words
    removed_words = [word for word in words if word.lower() in custom_stop_words]
    cleaned_words = [word for word in words if word.lower() not in custom_stop_words]
    cleaned_text = ' '.join(cleaned_words)
    return cleaned_text


In [67]:
# Apply the cleaning function to the "text" column
df["cleaned_text"] = df["text"].apply(lambda x: clean_text_with_custom_stop_words(x, custom_stop_words))

In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 102 entries, 0 to 113
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   URL_ID        102 non-null    float64
 1   URL           102 non-null    object 
 2   text          102 non-null    object 
 3   cleaned_text  102 non-null    object 
dtypes: float64(1), object(3)
memory usage: 4.0+ KB


In [21]:
df.head()

URL_ID  \
0   123.0   
1   321.0   
3  4321.0   
4   432.0   
5  2893.8   

                                                                                                      URL  \
0         https://insights.blackcoffer.com/rise-of-telemedicine-and-its-impact-on-livelihood-by-2040-3-2/   
1            https://insights.blackcoffer.com/rise-of-e-health-and-its-impact-on-humans-by-the-year-2030/   
3           https://insights.blackcoffer.com/rise-of-telemedicine-and-its-impact-on-livelihood-by-2040-2/   
4         https://insights.blackcoffer.com/rise-of-telemedicine-and-its-impact-on-livelihood-by-2040-2-2/   
5  https://insights.blackcoffer.com/rise-of-chatbots-and-its-impact-on-customer-support-by-the-year-2040/   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     

In [17]:
! unzip "/content/MasterDictionary-20230806T061202Z-001.zip"

Archive:  /content/MasterDictionary-20230806T061202Z-001.zip
  inflating: MasterDictionary/negative-words.txt  
  inflating: MasterDictionary/positive-words.txt  


In [68]:
# Load the Master Dictionary
master_dictionary_folder = "/content/MasterDictionary"
positive_words = set()
negative_words = set()

for filename in os.listdir(master_dictionary_folder):
    file_path = os.path.join(master_dictionary_folder, filename)
    with open(file_path, 'r', encoding='latin-1') as file:
        words = file.read().splitlines()
        for word in words:
            if word.lower() not in custom_stop_words:  # Check against your custom stop words
                if filename.lower().startswith("positive"):
                    positive_words.add(word.lower())
                elif filename.lower().startswith("negative"):
                    negative_words.add(word.lower())

In [69]:
# SENTIMENT ANALYSIS
import nltk
from nltk import word_tokenize
# Create a new column to store positive scores
df["POSITIVE SCORE"] = 0
# Create a new column to store negative scores
df["NEGATIVE SCORE"] = 0
# Create a new column to store polarity scores
df["POLARITY SCORE"] = 0.0
# Create a new column to store subjectivity scores
df["SUBJECTIVITY SCORE"] = 0.0
# Loop through the dataframe rows and calculate positive scores
for index, row in df.iterrows():
    cleaned_text = row['cleaned_text']
    tokens = word_tokenize(cleaned_text)  # Tokenize the cleaned text
    # POSITIVE SCORE
    positive_score = sum(1 for token in tokens if token.lower() in positive_words)
    # NEGATIVE SCORE
    negative_score = sum(-1 for token in tokens if token.lower() in negative_words) * -1
    # POLARITY SCORE
    polarity_score = (positive_score - negative_score) / ((positive_score + negative_score) + 0.000001)
    total_words = len(cleaned_text.split())  # Count total words after cleaning
    #SUBJECTIVITY SCORE
    subjectivity_score = (positive_score + negative_score) / (total_words + 0.000001)

    df.at[index, "POSITIVE SCORE"] = positive_score
    df.at[index, "NEGATIVE SCORE"] = negative_score
    df.at[index, "POLARITY SCORE"] = polarity_score
    df.at[index, "SUBJECTIVITY SCORE"] = subjectivity_score

In [70]:
df.head()

URL_ID  \
0   123.0   
1   321.0   
3  4321.0   
4   432.0   
5  2893.8   

                                                                                                      URL  \
0         https://insights.blackcoffer.com/rise-of-telemedicine-and-its-impact-on-livelihood-by-2040-3-2/   
1            https://insights.blackcoffer.com/rise-of-e-health-and-its-impact-on-humans-by-the-year-2030/   
3           https://insights.blackcoffer.com/rise-of-telemedicine-and-its-impact-on-livelihood-by-2040-2/   
4         https://insights.blackcoffer.com/rise-of-telemedicine-and-its-impact-on-livelihood-by-2040-2-2/   
5  https://insights.blackcoffer.com/rise-of-chatbots-and-its-impact-on-customer-support-by-the-year-2040/   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     

In [71]:
# Create new columns to store analysis results
df["AVG SENTENCE LENGTH"] = 0.0
df["AVG SENTENCE LENGTH"] = 0.0
df["FOG INDEX"] = 0.0

def count_syllables(word):
    vowels = "aeiouAEIOU"
    count = 0
    prev_char = None
    for char in word:
        if char in vowels and (prev_char is None or prev_char not in vowels):
            count += 1
        prev_char = char
    if word.endswith(("es", "ed")):
        count -= 1
    if count == 0:
        count = 1  # Each word should have at least one syllable
    return count

# Loop through the dataframe rows and calculate analysis results
for index, row in df.iterrows():
    total_words = len(word_tokenize(row["cleaned_text"]))
    total_sentences = len(sent_tokenize(row["cleaned_text"]))
    # Check if total_sentences is zero to avoid division by zero
    if total_sentences == 0:
        average_sentence_length = 0
    else:
        average_sentence_length = total_words / total_sentences
    df.at[index, "AVG SENTENCE LENGTH"] = average_sentence_length

    complex_words = 0
    tokens = word_tokenize(row["cleaned_text"])
    for token in tokens:
        syllable_count = count_syllables(token)
        if syllable_count > 2:  # Count words with more than two syllables as complex words
            complex_words += 1
    percentage_complex_words = complex_words / total_words
    df.at[index, "AVG SENTENCE LENGTH"] = percentage_complex_words

    fog_index = 0.4 * (average_sentence_length + percentage_complex_words)
    df.at[index, "FOG INDEX"] = fog_index

In [72]:
df.head()

URL_ID  \
0   123.0   
1   321.0   
3  4321.0   
4   432.0   
5  2893.8   

                                                                                                      URL  \
0         https://insights.blackcoffer.com/rise-of-telemedicine-and-its-impact-on-livelihood-by-2040-3-2/   
1            https://insights.blackcoffer.com/rise-of-e-health-and-its-impact-on-humans-by-the-year-2030/   
3           https://insights.blackcoffer.com/rise-of-telemedicine-and-its-impact-on-livelihood-by-2040-2/   
4         https://insights.blackcoffer.com/rise-of-telemedicine-and-its-impact-on-livelihood-by-2040-2-2/   
5  https://insights.blackcoffer.com/rise-of-chatbots-and-its-impact-on-customer-support-by-the-year-2040/   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     

In [73]:
# Loop through the dataframe rows and calculate analysis results
for index, row in df.iterrows():
    # Calculate total words
    total_words = len(word_tokenize(row["cleaned_text"]))

    # Calculate total sentences
    total_sentences = len(sent_tokenize(row["cleaned_text"]))

    # Calculate average number of words per sentence
    avg_words_per_sentence = total_words / total_sentences if total_sentences > 0 else 0
    df.at[index, "AVG NUMBER OF WORDS PER SENTENCE"] = avg_words_per_sentence

    # Calculate complex word count
    complex_words = 0
    tokens = word_tokenize(row["cleaned_text"])
    for token in tokens:
        syllable_count = count_syllables(token)
        if syllable_count > 2:  # Count words with more than two syllables as complex words
            complex_words += 1
    df.at[index, "COMPLEX WORD COUNT"] = complex_words


In [74]:
from nltk.corpus import stopwords
import string

# Load the NLTK stop words
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

# Function to calculate word count
def calculate_word_count(text):
    words = text.split()
    cleaned_words = [word for word in words if word.lower() not in stop_words]
    cleaned_words = [word.strip(string.punctuation) for word in cleaned_words]
    cleaned_words = [word for word in cleaned_words if word]  # Remove empty words
    word_count = len(cleaned_words)
    return word_count

# Loop through the dataframe rows and calculate word count
df["WORD COUNT"] = df["cleaned_text"].apply(calculate_word_count)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [75]:
def count_syllables2(word):
    vowels = "aeiouAEIOU"
    count = 0
    prev_char = None
    for char in word:
        if char in vowels and (prev_char is None or prev_char not in vowels):
            count += 1
        prev_char = char
    if word.endswith(("es", "ed")):
        count -= 1
    if count == 0:
        count = 1  # Each word should have at least one syllable
    return count

# Function to calculate syllable count per word
def calculate_syllable_count(text):
    words = text.split()
    syllable_counts = [count_syllables(word) for word in words]
    return syllable_counts

# Loop through the dataframe rows and calculate syllable count per word
df["SYLLABLE PER WORD"] = df["cleaned_text"].apply(calculate_syllable_count)


In [76]:

# Function to calculate count of personal pronouns
def calculate_personal_pronouns(text):
    pronoun_pattern = re.compile(r'\b(I|we|my|ours|us)\b', flags=re.IGNORECASE)
    personal_pronouns = pronoun_pattern.findall(text)
    count = len(personal_pronouns)
    return count

# Loop through the dataframe rows and calculate personal pronoun count
df["PERSONAL PRONOUNS"] = df["cleaned_text"].apply(calculate_personal_pronouns)


In [77]:
for index, row in df.iterrows():
    text = row["cleaned_text"]
    tokens = word_tokenize(text)
    total_words = len(tokens)

    # Calculate average word length
    cleaned_words = [word for word in tokens if word.lower() not in custom_stop_words and word.isalpha()]
    total_characters = sum(len(word) for word in cleaned_words)
    avg_word_length = total_characters / len(cleaned_words) if len(cleaned_words) > 0 else 0
    df.at[index, "AVG WORD LENGTH"] = avg_word_length

In [78]:
df.head()

URL_ID  \
0   123.0   
1   321.0   
3  4321.0   
4   432.0   
5  2893.8   

                                                                                                      URL  \
0         https://insights.blackcoffer.com/rise-of-telemedicine-and-its-impact-on-livelihood-by-2040-3-2/   
1            https://insights.blackcoffer.com/rise-of-e-health-and-its-impact-on-humans-by-the-year-2030/   
3           https://insights.blackcoffer.com/rise-of-telemedicine-and-its-impact-on-livelihood-by-2040-2/   
4         https://insights.blackcoffer.com/rise-of-telemedicine-and-its-impact-on-livelihood-by-2040-2-2/   
5  https://insights.blackcoffer.com/rise-of-chatbots-and-its-impact-on-customer-support-by-the-year-2040/   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     

In [79]:
cols_drop=['text', 'cleaned_text']
df.drop(cols_drop, axis=1, inplace=True)

In [80]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 102 entries, 0 to 113
Data columns (total 14 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   URL_ID                            102 non-null    float64
 1   URL                               102 non-null    object 
 2   POSITIVE SCORE                    102 non-null    int64  
 3   NEGATIVE SCORE                    102 non-null    int64  
 4   POLARITY SCORE                    102 non-null    float64
 5   SUBJECTIVITY SCORE                102 non-null    float64
 6   AVG SENTENCE LENGTH               102 non-null    float64
 7   FOG INDEX                         102 non-null    float64
 8   AVG NUMBER OF WORDS PER SENTENCE  102 non-null    float64
 9   COMPLEX WORD COUNT                102 non-null    float64
 10  WORD COUNT                        102 non-null    int64  
 11  SYLLABLE PER WORD                 102 non-null    object 
 12  PERSONAL

In [82]:
df.to_csv("output.csv")

In [81]:
df.head()

,URL_ID,URL,POSITIVE SCORE,NEGATIVE SCORE,POLARITY SCORE,SUBJECTIVITY SCORE,AVG SENTENCE LENGTH,FOG INDEX,AVG NUMBER OF WORDS PER SENTENCE,COMPLEX WORD COUNT,WORD COUNT,SYLLABLE PER WORD,PERSONAL PRONOUNS,AVG WORD LENGTH
0,123.0,https://insights.blackcoffer.com/rise-of-telemedicine-and-its-impact-on-livelihood-by-2040-3-2/,76,23,0.535354,0.116197,0.442308,4.634066,11.142857,414.0,830,"[2, 6, 2, 4, 1, 3, 2, 6, 3, 3, 2, 3, 2, 2, 1, 2, 3, 2, 2, 4, 1, 3, 3, 3, 2, 6, 3, 2, 2, 3, 3, 2, 6, 3, 3, 3, 2, 3, 2, 1, 4, 4, 5, 3, 3, 3, 2, 2, 2, 3, 1, 1, 2, 6, 3, 6, 3, 4, 6, 3, 2, 5, 3, 2, 2, 6, 2, 3, 4, 2, 1, 1, 2, 2, 2, 1, 6, 2, 3, 1, 3, 3, 3, 3, 1, 2, 3, 2, 2, 4, 2, 2, 3, 2, 3, 4, 3, 2, 3, 2, ...]",0,7.731824
1,321.0,https://insights.blackcoffer.com/rise-of-e-health-and-its-impact-on-humans-by-the-year-2030/,38,13,0.490196,0.168317,0.464832,5.635933,13.625000,152.0,297,"[2, 1, 1, 2, 2, 1, 3, 2, 2, 1, 1, 4, 5, 1, 2, 1, 4, 3, 2, 2, 1, 3, 3, 2, 3, 2, 3, 3, 6, 3, 3, 1, 1, 1, 1, 4, 2, 3, 3, 3, 2, 3, 3, 2, 1, 1, 2, 1, 3, 1, 1, 2, 2, 3, 3, 6, 3, 2, 3, 2, 3, 2, 3, 3, 4, 2, 2, 3, 2, 2, 3, 3, 3, 3, 1, 4, 2, 3, 2, 1, 3, 3, 2, 1, 6, 3, 2, 3, 3, 3, 2, 1, 3, 3, 3, 3, 2, 2, 2, 3, ...]",0,7.713793
3,4321.0,https://insights.blackcoffer.com/rise-of-telemedicine-and-its-impact-on-livelihood-by-2040-2/,34,26,0.133333,0.089686,0.347645,5.487206,13.370370,251.0,641,"[2, 6, 2, 4, 1, 3, 2, 1, 2, 5, 5, 2, 1, 1, 2, 2, 3, 1, 2, 2, 4, 6, 1, 3, 2, 1, 1, 2, 4, 2, 3, 3, 2, 4, 2, 3, 2, 3, 3, 2, 1, 3, 2, 3, 1, 3, 1, 2, 4, 2, 2, 6, 7, 3, 2, 3, 4, 3, 4, 2, 3, 2, 3, 2, 3, 3, 2, 2, 3, 2, 3, 1, 3, 3, 2, 3, 3, 2, 2, 1, 1, 3, 6, 1, 3, 2, 3, 2, 3, 3, 4, 3, 3, 2, 4, 1, 4, 3, 4, 2, ...]",0,7.215071
4,432.0,https://insights.blackcoffer.com/rise-of-telemedicine-and-its-impact-on-livelihood-by-2040-2-2/,34,26,0.133333,0.089686,0.347645,5.487206,13.370370,251.0,641,"[2, 6, 2, 4, 1, 3, 2, 1, 2, 5, 5, 2, 1, 1, 2, 2, 3, 1, 2, 2, 4, 6, 1, 3, 2, 1, 1, 2, 4, 2, 3, 3, 2, 4, 2, 3, 2, 3, 3, 2, 1, 3, 2, 3, 1, 3, 1, 2, 4, 2, 2, 6, 7, 3, 2, 3, 4, 3, 4, 2, 3, 2, 3, 2, 3, 3, 2, 2, 3, 2, 3, 1, 3, 3, 2, 3, 3, 2, 2, 1, 1, 3, 6, 1, 3, 2, 3, 2, 3, 3, 4, 3, 3, 2, 4, 1, 4, 3, 4, 2, ...]",0,7.215071
5,2893.8,https://insights.blackcoffer.com/rise-of-chatbots-and-its-impact-on-customer-support-by-the-year-2040/,43,11,0.592593,0.096601,0.387722,4.214105,10.147541,240.0,543,"[2, 2, 2, 3, 2, 1, 3, 2, 2, 3, 3, 3, 2, 4, 5, 3, 4, 2, 3, 2, 2, 1, 4, 3, 3, 4, 5, 3, 2, 1, 2, 2, 3, 3, 1, 3, 2, 4, 1, 1, 5, 4, 1, 2, 1, 2, 3, 1, 1, 3, 3, 2, 4, 1, 1, 1, 2, 2, 3, 4, 4, 2, 5, 5, 4, 3, 3, 3, 2, 1, 4, 3, 3, 3, 4, 1, 2, 1, 3, 1, 3, 4, 3, 4, 2, 2, 1, 1, 3, 5, 3, 3, 3, 2, 2, 1, 2, 4, 3, 3, ...]",2,7.446097
